In [1]:
from config import config_dict_imbalanced

# document config order
CONFIG = config_dict_imbalanced["CONFIG-I55"]
print('Proceed with configuration:', CONFIG["NAME"])

Proceed with configuration: miniloan


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
# import os
from sklearn.model_selection import train_test_split #, GridSearchCV
from sklearn.metrics import matthews_corrcoef,fbeta_score,confusion_matrix,f1_score,precision_score, recall_score, accuracy_score, balanced_accuracy_score, confusion_matrix, r2_score, explained_variance_score, mean_absolute_error, max_error
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
import wittgenstein as lw
import time
import warnings
import re
from aix360.algorithms.rbm import BRCGExplainer, BooleanRuleCG, GLRMExplainer, LinearRuleRegression
if CONFIG['BINARIZER'] == 'QUANTILE':
    from aix360.algorithms.rbm import FeatureBinarizer
elif CONFIG['BINARIZER'] == 'TREES':
    from aix360.algorithms.rbm import FeatureBinarizerFromTrees
if CONFIG['ALGO'] == 'RIPPER':
    from aix360i.algorithms.rule_induction.ripper import Ripper
if CONFIG['ALGO'] == 'CORELS':
    from corels import *

Importing dev version v0.981 of RIPPER


In [3]:
def convert(char):
    if char == CONFIG['POS_CLASS']:
        return 1
    else:
        return 0

df = pd.read_csv(CONFIG['DATA_SET'],dtype=CONFIG['DATA_TYPES'])
df = df.drop(columns=CONFIG['DROP'])
if CONFIG['ALGO'] == 'BRCG' or CONFIG['ALGO'] == 'CORELS' :
    df[CONFIG['TARGET_LABEL']] = df[CONFIG['TARGET_LABEL']].map(convert)
    CONFIG['POS_CLASS'] = 1
    # maybe this could also be achieved through explicit binarization of target vector
df.info()
df[CONFIG['TARGET_LABEL']].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   creditScore          100000 non-null  float64
 1   income               100000 non-null  float64
 2   loanAmount           100000 non-null  float64
 3   monthDuration        100000 non-null  float64
 4   rate                 100000 non-null  float64
 5   approval             100000 non-null  bool   
 6   yearlyReimbursement  100000 non-null  float64
dtypes: bool(1), float64(6)
memory usage: 4.7 MB


True     78046
False    21954
Name: approval, dtype: int64

In [4]:
if CONFIG['UNDERSAMP'] == True:
    under_sampling_fraud = df[df[CONFIG['TARGET_LABEL']] == 1]
    under_size = under_sampling_fraud.size
    under_sampling_legit = (df[df[CONFIG['TARGET_LABEL']] == 0].sample(n=4000, random_state = 42))
    df = pd.concat([under_sampling_legit, under_sampling_fraud])

In [5]:
if CONFIG["ONEHOT"] == True:
    from collections import defaultdict
    from sklearn.preprocessing import OneHotEncoder, LabelEncoder
    categorial_feat = df.select_dtypes(include=['object']).columns
    numercial_feat = df.select_dtypes(include=['int64', 'float64']).columns

    def one_hot_encode_category(df, categorial_index):
        d = defaultdict(LabelEncoder)
        # Encoding
        lecatdata = df[categorial_index].apply(lambda x: d[x.name].fit_transform(x))
        for x in range(len(categorial_index)):
            print(categorial_index[x],": ", df[categorial_index[x]].unique())
            print(categorial_index[x],": ", lecatdata[categorial_index[x]].unique())
        #One hot encoding with dummy variable
        dummyvars = pd.get_dummies(df[categorial_index])

        return dummyvars

    dummyvars =one_hot_encode_category(df,categorial_feat )
    df = pd.concat([df[numercial_feat], dummyvars], axis = 1)
    df

In [6]:
if CONFIG['TRAIN_TEST_SPLIT'] == 'FIXED':
    if CONFIG['MODE'] == 'PREDICTIVE':
        train = df[df['is_test_set'] == False]
        test = df[df['is_test_set'] == True]
    elif CONFIG['MODE'] == 'DESCRIPTIVE':
        train = df
        test = df

    train = train.drop(columns=['is_test_set'])
    test = test.drop(columns=['is_test_set'])

    y_train = train[CONFIG['TARGET_LABEL']]
    x_train = train.drop(columns=[CONFIG['TARGET_LABEL']])

    y_test = test[CONFIG['TARGET_LABEL']]
    x_test = test.drop(columns=[CONFIG['TARGET_LABEL']])
else:
    x_train, x_test, y_train, y_test = train_test_split(df.drop(columns=[CONFIG['TARGET_LABEL']]), df[CONFIG['TARGET_LABEL']], test_size=CONFIG['TRAIN_TEST_SPLIT'], random_state=42)

print('Training:', x_train.shape, y_train.shape)
print('Test:', x_test.shape, y_test.shape)

Training: (57432, 6) (57432,)
Test: (24614, 6) (24614,)


In [7]:
if CONFIG['BINARIZER'] == 'TREES':
    binarizer =  FeatureBinarizerFromTrees(negations=False, randomState=42) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train, y_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)
elif CONFIG['BINARIZER'] == 'QUANTILE':
    binarizer =  FeatureBinarizer(numThresh=9,negations=False, randomState=42) # FeatureBinarizer(negations=False), FeatureBinarizerFromTrees(negations=True, randomState=42)
    binarizer = binarizer.fit(x_train)
    x_train_bin = binarizer.transform(x_train) #  x_train_bin = binarizer.fit_transform(x_train)
    x_test_bin = binarizer.transform(x_test) #  X_fb = self.fb.fit_transform(X_train)  
elif CONFIG['BINARIZER'] == 'NATIVE':
    x_train_bin = x_train
    x_test_bin = x_test
else:
    print('UNRECOGNIZED BINARIZER')

x_train_bin.info() # verbose=True
x_train_bin.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57432 entries, 70812 to 15066
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   creditScore          57432 non-null  float64
 1   income               57432 non-null  float64
 2   loanAmount           57432 non-null  float64
 3   monthDuration        57432 non-null  float64
 4   rate                 57432 non-null  float64
 5   yearlyReimbursement  57432 non-null  float64
dtypes: float64(6)
memory usage: 3.1 MB


,creditScore,income,loanAmount,monthDuration,rate,yearlyReimbursement
70812,576.0,93600.0,1009583.0,313.0,0.051843,70673.0
63636,543.0,293500.0,1188676.0,155.0,0.040505,118341.0
82740,718.0,199395.0,1717330.0,257.0,0.040763,120345.0
44315,793.0,134015.0,1505656.0,246.0,0.051968,119532.0
90832,531.0,270293.0,1246568.0,234.0,0.030156,84632.0


In [8]:
#XGboost for Binary Classification
start_time = time.time()
    
if CONFIG['TYPE'] == 'BINARY':
    
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    x_train_bin.columns = [regex.sub("_", str(col)) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_train_bin.columns.values]
    x_test_bin.columns = [regex.sub("_", str(col)) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_test_bin.columns.values]

    xgb_cl = xgb.XGBClassifier()
    xgb_cl.fit(x_train_bin, y_train)
    end_time = time.time()
    preds = xgb_cl.predict(x_test_bin)     
    print('Accuracy:', accuracy_score(y_test, preds))
    print('Balanced accuracy:', balanced_accuracy_score(y_test, preds))
    print('Precision:', precision_score(y_test, preds, pos_label=CONFIG['POS_CLASS']))
    print('Recall:', recall_score(y_test, preds, pos_label=CONFIG['POS_CLASS']))
    print('F-2', fbeta_score(y_test, preds, pos_label=CONFIG['POS_CLASS'], beta= 2))
    print('ConfusionMatrix', confusion_matrix(y_test, preds))
    print('Mathhews', matthews_corrcoef(y_test, preds))
    print('Training time: ' + str(end_time - start_time))
   

Accuracy: 0.9986999268708865
Balanced accuracy: 0.9874364260443182
Precision: 0.9982905982905983
Recall: 0.9749582637729549
F-2 0.9795370680979537
ConfusionMatrix [[ 1168    30]
 [    2 23414]]
Mathhews 0.9858797652295478
Training time: 0.7819991111755371


In [9]:
df[CONFIG["TARGET_LABEL"]].value_counts()

True     78046
False     4000
Name: approval, dtype: int64